In [16]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import albumentations as A
import os

Чтение текстового файла с именами

In [17]:
def get_image_names(image_name_txt):
    filenames = open(image_name_txt, 'r')
    content = filenames.read()
    content_list = content.split("\n")
    
    return content_list

names_txt_path = 'E:/Dataset_images/valid_names.txt'
image_names = get_image_names(names_txt_path)
print(image_names)

['F_06_SER_1_12', 'F_10_SER_11_17', 'F_01_SER_3_2', 'F_09_SER_1_8', 'F_09_SER_3_15', 'F_07_SER_1_0', 'F_05_SER_5_9', 'F_02_SER_6_21', 'F_05_SER_5_2', 'F_04_SER_4_13', 'F_05_SER_1_7', 'F_10_SER_2_0', 'F_03_SER_4_0', 'F_03_SER_6_21', 'F_11_SER_8_5', 'F_08_SER_8_7', 'F_07_SER_6_8', 'F_04_SER_2_22', 'F_11_SER_9_12', 'F_02_SER_7_0', 'F_02_SER_6_5', 'F_05_SER_4_12', 'F_02_SER_7_9', 'F_10_SER_7_6', 'F_05_SER_2_10', 'F_03_SER_8_24', 'F_10_SER_7_12', 'F_02_SER_4_3', 'F_03_SER_9_2', 'F_06_SER_3_11', 'F_11_SER_6_4', 'F_01_SER_2_1', 'F_11_SER_9_16', 'F_04_SER_3_5', 'F_08_SER_8_12', 'F_10_SER_10_1', 'F_10_SER_3_4', 'F_06_SER_2_13', 'F_02_SER_5_27', 'F_07_SER_1_6', 'F_03_SER_8_5', 'F_06_SER_2_19', 'F_06_SER_5_1', 'F_04_SER_1_4', 'F_11_SER_5_20', 'F_08_SER_1_5', 'F_04_SER_1_8', 'F_10_SER_10_9', 'F_03_SER_3_2', 'F_03_SER_8_3', 'F_07_SER_5_18', 'F_03_SER_3_1', 'F_08_SER_7_2', 'F_10_SER_9_8', 'F_02_SER_10_2', 'F_02_SER_1_0', 'F_05_SER_1_17', 'F_06_SER_5_3', 'F_02_SER_7_5', 'F_03_SER_2_16', 'F_02_SER_2_1

In [18]:
def norm_label(label):
    thresh = 0
    fn = lambda x : 255 if x > thresh else 0
    label = label.convert('L').point(fn, mode='1')
    return label

Функция аугментации

In [19]:
def apply_transform(transform, image_dir, image_name, label_dir, postfix, save_image_dir, save_label_dir, names_txt_path, label_transform = None):
    image_path = os.path.join(image_dir, image_name+'.png')
    label_path = os.path.join(label_dir, image_name+'_lab.png')
    
    image = Image.open(image_path)
    label = Image.open(label_path)
    image_np = np.array(image)
    image_np = image_np.astype(np.uint16)
    label_np = np.array(label)
    label_np = label_np.astype(np.uint16)
    
    transform_result = transform(image=image_np, mask = label_np)
    image_transformed = transform_result['image']
    label_transformed = transform_result['mask']
    
    image_transformed = image_transformed.astype(np.uint32)
    label_transformed = label_transformed.astype(np.uint32)
    
    image_augmentated = Image.fromarray(image_transformed)
    label_augmentated = Image.fromarray(label_transformed)
    
    label_augmentated = norm_label(label_augmentated)
    
    image_name_save = image_name + postfix + '.png'
    label_name_save = image_name + postfix + '_lab.png'
    
    image_path_save = os.path.join(save_image_dir, image_name_save)
    label_path_save = os.path.join(save_label_dir, label_name_save)
    
    image_augmentated.save(image_path_save)
    label_augmentated.save(label_path_save)
    
    with open(names_txt_path, 'a') as f:
        f.write("{0:}\n".format(image_name_save[:-4]))
    

In [20]:
transform = A.RandomBrightnessContrast(brightness_limit=[-0.1,0], brightness_by_max=True, p=1)

In [21]:
image_dir = 'E:/Dataset_images/images'
label_dir = 'E:/Dataset_images/labels'
save_image_dir = 'E:/augmentated_data_new/images'
save_label_dir = 'E:/augmentated_data_new/labels'
postfix ='_br'
names_txt_path = 'E:/augmentated_data_new/aug_valid.txt'
for image_name in image_names:
    apply_transform(transform, image_dir, image_name, label_dir, postfix, save_image_dir, save_label_dir, names_txt_path)